In [1]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise
from scipy.spatial import distance_matrix
from scipy.optimize import linear_sum_assignment
import random

In [2]:
def threshold(x, img):
    # Convert into binary image using thresholding
    # Documentation for threshold: http://docs.opencv.org/modules/imgproc/doc/miscellaneous_transformations.html?highlight=threshold#threshold
    # Example of thresholding: http://docs.opencv.org/doc/tutorials/imgproc/threshold/threshold.html
    thres_output = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,x,-5)
    # Find contours
	# Documentation for finding contours: http://docs.opencv.org/modules/imgproc/doc/structural_analysis_and_shape_descriptors.html?highlight=findcontours#findcontours
    contours, hierarchy = cv2.findContours(thres_output, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    return contours, thres_output

In [3]:
def my_frame_differencing(prev, curr):
    '''
    Function that does frame differencing between the current frame and the previous frame
    Args:
        src The current color image
        prev The previous color image
    Returns:
        dst The destination grayscale image where pixels are colored white if the corresponding pixel intensities in the current
    and previous image are not the same
    '''
    dst = cv2.absdiff(prev, curr)
    gs = cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY)
    dst = (gs > 20).astype(np.uint8) * 255
    return dst

In [4]:
# find centers of the detected contours, return list of centers
def find_center(contours):
    centers = []
    for c in contours:
        M = cv2.moments(c)
        if M['m00'] == 0:
            cX = int(c[0][0][0])
            cY = int(c[0][0][1])
            centers.append([cX,cY])
        else:
            cX = int(M['m10']/M['m00'])
            cY = int(M['m01']/M['m00'])
            centers.append([cX,cY])
    return centers

In [5]:
def area(countours,thresh):
    temp = []
    for i in range(len(contours)):
        (x,y,w,h) = cv2.boundingRect(contours[i])
        if w*h >=thresh:
            temp.append(contours[i])
    return temp

In [6]:
# find match between the list of previous center positions and current center positions
# return two list of matching indexes: p_ind, c_ind where p_ind[0] matches with c_ind[0]
def find_match(prev,cur,thresh):
    # calculate pairwise distance and stores into a matrix
    cost = distance_matrix(prev,cur) 
    cost[cost>thresh]=9999
    
    # use the distance matrix as the cost to do the matching using Hungarian algorithm which
    # minimizes the total cost while each row or column has at most one match
    p_ind, c_ind = linear_sum_assignment(cost) 
    return p_ind,c_ind

In [7]:
# Define each item
class track:
    def __init__(self, position):
        self.color = (random.randint(50,255), random.randint(50,255), random.randint(50,255))
        self.track = [tuple(position)]
        self.pos = position 
        # initialize kf for the specific track
        self.kf = KalmanFilter(dim_x=4, dim_z=2)
        self.kf.x = position+[0,0] #initial value
        self.kf.F = np.array([[1,0,1,0], [0,1,0,1], [0,0,1,0], [0,0,0,1]]) # linear mapping from one state to the next
        self.kf.H = np.array([[1,0,0,0], [0,1,0,0]]) # linear mapping from prediction to observations
        self.kf.P = 0.001 * np.eye(4) # estimate covariance matrix
        self.kf.Q = 0.001 * np.eye(4) # Process uncertainty/noise
        self.kf.R = 0.001 * np.eye(2) # measurement uncertainty/noise
    
    def predict(self):
        self.kf.predict()
        return self.kf.x.astype(int)[:2]

        
    def update(self,position):
        self.kf.update(position)
        self.pos = self.kf.x.astype(int)[:2].tolist()
        self.track += [tuple(self.pos)]


In [8]:
def distance(x,y):
    return np.sqrt((x[1]-y[1])**2 + (x[0]-y[0])**2)

In [9]:
# bat images
img_path = './bats/Gray'
files = [join(img_path,f) for f in listdir(img_path) if isfile(join(img_path,f))]
files.sort()
img = []
for i in range(len(files)):
    img.append(cv2.imread(files[i]))

In [10]:
img_indx = 1
while img_indx < len(files):
# while img_indx < 50:
    cur = img[img_indx]
    gray = cv2.cvtColor(cur, cv2.COLOR_BGR2GRAY)
    blur = cv2.blur(gray, (5, 5))
    contours, thres_output = threshold(105, blur)
    contours = area(contours,71)
    output = img[img_indx][:,:,:].copy()
    

    if img_indx == 1:
        centers = find_center(contours)
        tracks = [track(i) for i in centers]
    else:
        prev = [i.predict() for i in tracks]
        current = find_center(contours)
        p_ind,c_ind = find_match(prev,current,80)
        for i in range(len(p_ind)):
            if distance(tracks[p_ind[i]].pos,current[c_ind[i]]) > 100:
#                 print('dist too large')
                tracks.append(track(current[c_ind[i]]))
            else:  
                tracks[p_ind[i]].update(current[c_ind[i]])
        
        for i in range(len(current)):
#             print('not in c_ind')
            if i not in c_ind:
                tracks.append(track(current[i]))
        
    for i in range(len(contours)):
        boundrec = cv2.boundingRect(contours[i])
        cv2.rectangle(output, boundrec, (0, 255, 0), 1, 8, 0)
            
    if (len(tracks) > 0):
        for i in range(len(tracks)):
            if len(tracks[i].track)>1:
                for j in range(len(tracks[i].track)-1):
                    cv2.line(output,tracks[i].track[j],tracks[i].track[j+1],tracks[i].color,1)

        # Show in a window
        cv2.namedWindow("Tracks", cv2.WINDOW_AUTOSIZE)
        cv2.imshow("Tracks", output)
        if cv2.waitKey(100)&0xFF == 27:
            break
    
    img_indx += 1

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [11]:
# cell images
img_path = './cells'
files = [join(img_path,f) for f in listdir(img_path) if isfile(join(img_path,f))]
files.sort()
img = []
for i in range(len(files)):
    img.append(cv2.imread(files[i]))

In [15]:
img_indx = 2
while img_indx < len(files):
# while img_indx < 20:
    prev = img[img_indx-1]
    cur = img[img_indx]
    
    diff = my_frame_differencing(prev, cur)
    
    cv2.namedWindow("Segmentation", cv2.WINDOW_AUTOSIZE)
    cv2.imshow("Segmentation", diff)
    
    size = 10
    element = np.ones((size,size))
    diff = cv2.dilate(diff, element)
    diff = cv2.dilate(diff, element)
    
    
    ret, thresh = cv2.threshold(diff,20,255,cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    contours = area(contours,2500)

    output = img[img_indx][:,:,:].copy()
    
    if img_indx-1 == 1:
        centers = find_center(contours)
        tracks = [track(i) for i in centers]
    else:
        prev = [i.predict() for i in tracks]
        current = find_center(contours)
        p_ind,c_ind = find_match(prev,current,100)
        for i in range(len(p_ind)):
            if distance(tracks[p_ind[i]].pos,current[c_ind[i]]) > 200:
                tracks.append(track(current[c_ind[i]]))
            else:  
                tracks[p_ind[i]].update(current[c_ind[i]])
        
        for i in range(len(current)):
            if i not in c_ind:
                tracks.append(track(current[i]))
        
    for i in range(len(contours)):
        boundrec = cv2.boundingRect(contours[i])
        cv2.rectangle(output, boundrec, (0, 255, 0), 1, 8, 0)
            
    if (len(tracks) > 0):
        for i in range(len(tracks)):
            if len(tracks[i].track)>1:
                for j in range(len(tracks[i].track)-1):
                    cv2.line(output,tracks[i].track[j],tracks[i].track[j+1],tracks[i].color,2)

        # Show in a window
        cv2.namedWindow("Tracks", cv2.WINDOW_AUTOSIZE)
        cv2.imshow("Tracks", output)
        if cv2.waitKey(100)&0xFF == 27:
            break
    
    img_indx += 1

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

-1